In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/06 17:33:33 WARN Utils: Your hostname, Ritikas-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.100 instead (on interface en0)
24/03/06 17:33:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/06 17:33:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/06 17:33:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
spark.version

'3.5.1'

In [19]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2019-10.parquet

--2024-03-06 17:15:41--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2019-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 2600:9000:26c0:dc00:b:20a5:b140:21, 2600:9000:26c0:8000:b:20a5:b140:21, 2600:9000:26c0:7e00:b:20a5:b140:21, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|2600:9000:26c0:dc00:b:20a5:b140:21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18344035 (17M) [application/x-www-form-urlencoded]
Saving to: ‘fhv_tripdata_2019-10.parquet’

fhv_tripdata_2019-1 100%[===================>]  17.49M  43.7MB/s    in 0.4s    

2024-03-06 17:15:41 (43.7 MB/s) - ‘fhv_tripdata_2019-10.parquet’ saved [18344035/18344035]



In [5]:
import pandas as pd

# Specify the path to your Parquet file
parquet_file_path = "fhv_tripdata_2019-10.parquet"

# Specify the path where you want to save the CSV file
csv_file_path = "fhv_tripdata_2019-10.csv"

# Read Parquet file into a Pandas DataFrame
df = pd.read_parquet(parquet_file_path, engine="pyarrow")  # Use "fastparquet" if you prefer

# Write the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)

print("Conversion completed successfully.")

Conversion completed successfully.


In [3]:
!wc -l fhv_tripdata_2019-10.csv

 1897857 fhv_tripdata_2019-10.csv


In [4]:
df = spark.read \
    .option("header", "true") \
    .csv('fhv_tripdata_2019-10.csv')

In [5]:
df.dtypes

[('dispatching_base_num', 'string'),
 ('pickup_datetime', 'string'),
 ('dropOff_datetime', 'string'),
 ('PUlocationID', 'string'),
 ('DOlocationID', 'string'),
 ('SR_Flag', 'string'),
 ('Affiliated_base_number', 'string')]

In [6]:
df.head(5)

[Row(dispatching_base_num='B00009', pickup_datetime='2019-10-01 00:23:00', dropOff_datetime='2019-10-01 00:35:00', PUlocationID='264.0', DOlocationID='264.0', SR_Flag=None, Affiliated_base_number='B00009'),
 Row(dispatching_base_num='B00013', pickup_datetime='2019-10-01 00:11:29', dropOff_datetime='2019-10-01 00:13:22', PUlocationID='264.0', DOlocationID='264.0', SR_Flag=None, Affiliated_base_number='B00013'),
 Row(dispatching_base_num='B00014', pickup_datetime='2019-10-01 00:11:43', dropOff_datetime='2019-10-01 00:37:20', PUlocationID='264.0', DOlocationID='264.0', SR_Flag=None, Affiliated_base_number='B00014'),
 Row(dispatching_base_num='B00014', pickup_datetime='2019-10-01 00:56:29', dropOff_datetime='2019-10-01 00:57:47', PUlocationID='264.0', DOlocationID='264.0', SR_Flag=None, Affiliated_base_number='B00014'),
 Row(dispatching_base_num='B00014', pickup_datetime='2019-10-01 00:23:09', dropOff_datetime='2019-10-01 00:28:27', PUlocationID='264.0', DOlocationID='264.0', SR_Flag=None,

In [7]:
from pyspark.sql import types

In [8]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [9]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhv_tripdata_2019-10.csv')

In [10]:
df.dtypes

[('dispatching_base_num', 'string'),
 ('pickup_datetime', 'timestamp'),
 ('dropoff_datetime', 'timestamp'),
 ('PULocationID', 'int'),
 ('DOLocationID', 'int'),
 ('SR_Flag', 'string'),
 ('Affiliated_base_number', 'string')]

In [11]:
df = df.repartition(6)

In [12]:
df

DataFrame[dispatching_base_num: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int, SR_Flag: string, Affiliated_base_number: string]

In [13]:
df.write.parquet('fhv/2019/10/')

In [14]:
df = spark.read.parquet('fhv/2019/10/')

In [24]:
import pandas as pd

file_path = "fhv_tripdata_2019-10.parquet"
df_main = spark.read.parquet(file_path)

In [15]:
from pyspark.sql import functions as F

In [16]:
#How many taxi trips were there on the 15th of October?

df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .filter("pickup_date = '2019-10-15'") \
    .count()

62629

In [18]:
df.registerTempTable('fhv_2019_10')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [19]:
spark.sql("""
SELECT
    COUNT(1)
FROM 
    fhv_2019_10
WHERE 
    to_date(pickup_datetime) = '2019-10-15';
;
""").show()

+--------+
|count(1)|
+--------+
|   62629|
+--------+



In [20]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [21]:
#Longest trip for each day

spark.sql("""
SELECT *
    FROM 
    fhv_2019_10
limit 5;
""").show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|     B03188         |2019-10-01 03:12:58|2019-10-01 04:24:16|        NULL|        NULL|   NULL|       B03188         |
|              B00887|2019-10-03 08:39:04|2019-10-03 09:22:27|        NULL|        NULL|   NULL|                B00887|
|              B00256|2019-10-02 12:23:48|2019-10-02 13:05:07|        NULL|        NULL|   NULL|                B00256|
|              B02834|2019-10-01 18:20:47|2019-10-01 18:58:47|        NULL|        NULL|   NULL|                B02834|
|     B01995         |2019-10-03 00:29:50|2019-10-03 00:35:53|        NULL|        NULL|   NULL|       B01995         |
+--------------------+------------------

In [22]:
spark.sql("""
SELECT
    to_date(pickup_datetime) AS pickup_date,
    MAX((CAST(dropoff_datetime AS LONG) - CAST(pickup_datetime AS LONG)) / 60) AS duration
FROM 
    fhv_2019_10
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 10;
""").show()

+-----------+-----------------+
|pickup_date|         duration|
+-----------+-----------------+
| 2019-10-28|       3.786915E7|
| 2019-10-11|       3.786915E7|
| 2019-10-31|       5260346.45|
| 2019-10-01|4207681.683333334|
| 2019-10-17|         527640.0|
| 2019-10-26|         527050.0|
| 2019-10-30|87932.06666666667|
| 2019-10-25|          63469.6|
| 2019-10-02|46213.88333333333|
| 2019-10-23|          44797.0|
+-----------+-----------------+



In [23]:
df \
    .withColumn('duration', df.dropoff_datetime.cast('long') - df.pickup_datetime.cast('long')) \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .groupBy('pickup_date') \
        .max('duration') \
    .orderBy('max(duration)', ascending=False) \
    .limit(5) \
    .show()

+-----------+-------------+
|pickup_date|max(duration)|
+-----------+-------------+
| 2019-10-28|   2272149000|
| 2019-10-11|   2272149000|
| 2019-10-31|    315620787|
| 2019-10-01|    252460901|
| 2019-10-17|     31658400|
+-----------+-------------+



In [25]:
df_main.registerTempTable('fhv_2019_10_main')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [26]:
spark.sql("""
SELECT
    * from fhv_2019_10_main
LIMIT 10;
""").show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|       264.0|       264.0|   NULL|                B00009|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|       264.0|       264.0|   NULL|                B00013|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|       264.0|       264.0|   NULL|                B00014|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|       264.0|       264.0|   NULL|                B00014|
|              B00014|2019-10-01 00:23:09|2019-10-01 00:28:27|       264.0|       264.0|   NULL|                B00014|
|     B00021         |2019-10-01 00:00:4

In [30]:
df_2 = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhv_tripdata_2019-10.csv')

In [31]:
df_2.write.parquet('fhv_2019_10/')

24/03/06 18:10:12 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/06 18:10:12 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/03/06 18:10:12 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/03/06 18:10:12 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/03/06 18:10:12 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
24/03/06 18:10:14 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/03/06 18:10:14 WARN MemoryManager: Total allocation exceeds 95.

In [32]:
df_2.registerTempTable('fhv_2019_10_2')

In [33]:
spark.sql("""
SELECT
    * from fhv_2019_10_2
LIMIT 10;
""").show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|        NULL|        NULL|   NULL|                B00009|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|        NULL|        NULL|   NULL|                B00013|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|        NULL|        NULL|   NULL|                B00014|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|        NULL|        NULL|   NULL|                B00014|
|              B00014|2019-10-01 00:23:09|2019-10-01 00:28:27|        NULL|        NULL|   NULL|                B00014|
|     B00021         |2019-10-01 00:00:4